In [1]:
import numpy as np
import random
from tqdm import tqdm
import math

In [2]:
def weight(x1,x2,x3,a,w=1): # 输入整个数组，选择的点来计算weight 变化
    S_x = (x3-x2)**2 / (2 * a * w) + a * w * (x3**2+x2**2)/4 + (x2-x1)**2 / (2 * a * w) + a * w* (x2**2+x1**2)/4
    W = np.exp(-S_x)
    return w

In [4]:
def generator(N_con,N_skip,N_dump,N,a,delta_u):
    U = np.zeros([N_con*N_skip+N_dump,N]) # 整个路径集
    x = np.zeros(N)
    y = x.copy()
    order = np.zeros(N)
    for i in range(len(order)):
        order[i] = i
    for i in tqdm(range (N_con*N_skip+N_dump)):
        random.shuffle(order)
        for k in range(N):
            j = int(order[k])
            y[j] = x[j] + delta_u * (1 - 2 * np.random.rand())
            if np.random.rand() <= weight(y[(j-1)%N],y[j],y[(j+1)%N],a)/weight(x[(j-1)%N],x[j],x[(j+1)%N],a):
                x[j] = y[j]
            else:
                y[j] = x[j]
        U[i,:] = x
    U_real = U[N_dump::N_skip,::] #去除未达到平衡的点并跳过相关联的点
    return U_real

In [5]:
N_con = 10000
N_skip = 50
N_dump = 1000
N = 1
a =0.1
delta_x = 0.5
U = generator(N_con,N_skip,N_dump,N,a,delta_x)
np.savetxt('Paths.txt',U,newline = '\n')
C_2 = np.zeros([25,N_con])
C_2std = np.zeros(25)
for i in tqdm(range(25)):
    for j in range(N_con):
        for k in range(N):
            C_2[i,j] = C_2[i,j] + U[j,k]*U[j,(k+i)%N]/N #对时间起点求平均
np.savetxt('C_2.txt', C_2.T,newline = '\n')

100%|██████████| 25/25 [00:00<00:00, 131.34it/s]


In [6]:
U_sqr = np.power(U,2)
x_2= np.mean(U_sqr) #基态能量
delta_x = np.std(U_sqr)/(N_con)**0.5
x_2,delta_x

(19862.981023709926, 124.10867419574613)

In [7]:
C_2 = np.zeros([25,N_con])

C_2std = np.zeros(25)
for i in tqdm(range(25)):
    for j in range(N_con):
        for k in range(N):
            C_2[i,j] = C_2[i,j] + U[j,k]*U[j,(k+i)%N]/N #对时间起点求平均
temp = C_2.copy()
C_2 = (np.sum(temp,axis = 0).reshape(1,25)-temp)/(N_con-1) #jackknife.
C_2mean = np.mean(C_2,axis = 0)
C_2std = np.std(C_2,axis = 0)*np.sqrt(N_con-1)
t = np.arange(25)
eff_m = np.log(C_2[:,:-1]/C_2[:,1:])/0.1

100%|██████████| 25/25 [00:00<00:00, 136.43it/s]


ValueError: cannot reshape array of size 10000 into shape (1,25)